In [290]:
import sympy as sp
import numpy as np
import time 

N = 3
g = sp.Symbol("g", real = True)

## Creating X coordinates (using sp.Matrix for the 4th stack)

In [139]:
# Creating a dictionary to store the X coordinates
X = {}
for i in np.arange(1,4):
    for j in np.arange(1,10):
        X[str(i)+str(j)] = sp.Symbol("X^{"+str(i)+"}_"+str(j), real = True)
            
for i in np.arange(1,10):
    #initialise a matrix of correct shape
    X_init = []
    for j in np.arange(1,N+1):
        row = []
        for k in np.arange(1,N+1):
            row.append(sp.Symbol("a_{"+str(j)+str(k)+"}"))
        X_init.append(row)
    X["4"+str(i)] = sp.Matrix(X_init)
    
    for j in np.arange(N):
        for k in np.arange(N):
            if j == k:
                X["4"+str(i)][j,k] = sp.Symbol("x^{4"+str(i)+"}_{"+str(j+1)+str(k+1)+"}",real = True)
            elif j > k:
                continue
            else:
                x_ij = sp.Symbol("x^{4"+str(i)+"}_{"+str(j+1)+str(k+1)+"}",real = True)
                y_ij = sp.Symbol("y^{4"+str(i)+"}_{"+str(j+1)+str(k+1)+"}",real = True)
                X["4"+str(i)][j,k] = x_ij + sp.I*y_ij
                X["4"+str(i)][k,j] = x_ij - sp.I*y_ij

## Creating X coordinates (using sp.MatrixSymbol for the 4th stack)

In [291]:
# Creating a dictionary to store the X coordinates
X = {}
for i in np.arange(1,5):
    for j in np.arange(1,10):
        if i != 4:
            X[str(i)+str(j)] = sp.Symbol("X^{"+str(i)+"}_"+str(j), real = True)
        else:
            X[str(i)+str(j)] = sp.MatrixSymbol("X^{"+str(i)+"}_"+str(j),N,N)

## Creating the $\phi$'s and $\phi^{\dagger}$'s and $\phi^{(ij)}$

In [292]:
phi = {}
phi_ = {}
phi_dag = {}
for i in np.arange(1,5):
    phi[str(i)+"1"] = 1/sp.sqrt(2) * (X[str(i)+"4"] + sp.I * X[str(i)+"5"])
    phi[str(i)+"2"] = 1/sp.sqrt(2) * (X[str(i)+"6"] + sp.I * X[str(i)+"7"])
    phi[str(i)+"3"] = 1/sp.sqrt(2) * (X[str(i)+"8"] + sp.I * X[str(i)+"9"])
    
    phi_dag[str(i)+"1"] = 1/sp.sqrt(2) * ( X[str(i)+"4"] - sp.I * X[str(i)+"5"])
    phi_dag[str(i)+"2"] = 1/sp.sqrt(2) * (X[str(i)+"6"] - sp.I * X[str(i)+"7"])
    phi_dag[str(i)+"3"] = 1/sp.sqrt(2) * (X[str(i)+"8"] - sp.I * X[str(i)+"9"])
    
phi_["12"] = phi["13"] - phi["23"]
phi_["23"] = phi["21"] - phi["31"]
phi_["31"] = phi["32"] - phi["12"]

## Creating Z coordinates

In [293]:
# Building the Z-dictionary
Z = {}
for i in np.arange(1,4):
    #Scalars here
    for j in np.arange(1,4):
        if i == j: 
            continue
        else:
            P = sp.Symbol("P^{"+str(i) + str(j) +"}", real = True)
            Q = sp.Symbol("Q^{"+str(i) + str(j) +"}", real = True)
            Z[str(i) + str(j)] = (P + sp.I * Q)
    
    #Vectors here
    store_current_4i = []
    store_current_i4 = []
    for j in np.arange(1,N+1):
        P4i = sp.Symbol("P^{"+str(4)+str(i)+"}_"+str(j), real = True)
        Q4i = sp.Symbol("Q^{"+str(4)+str(i)+"}_"+str(j), real = True)
        Pi4 = sp.Symbol("P^{"+str(i)+str(4)+"}_"+str(j), real = True)
        Qi4 = sp.Symbol("Q^{"+str(i)+str(4)+"}_"+str(j), real = True)
        store_current_4i.append([P4i + sp.I * Q4i])
        store_current_i4.append([Pi4 + sp.I * Qi4])
    Z["4"+str(i)] = sp.Matrix(store_current_4i)
    Z[str(i) + "4"] = sp.Matrix(store_current_i4).transpose()

## Creating the F variables

In [294]:
F = {}

#Scalars
for i in np.arange(1,4):
    for j in np.arange(1,5):
        if i == j:
            continue
        if j == 4:
            F[str(i)+str(j)] = (Z[str(i)+str(j)] * Z[str(j)+str(i)])[0] + N*sp.Symbol("c^{"+str(i)+str(j)+"}")/g**2
        else:
            F[str(i)+str(j)] = Z[str(i)+str(j)] * Z[str(j)+str(i)] + sp.Symbol("c^{"+str(i)+str(j)+"}")/g**2 

F["41"] = Z["41"]*Z["14"] + (sp.Symbol("c^{41}")/g**2)*sp.eye(N) +  phi["42"]*phi["43"] - phi["43"]*phi["42"]
F["42"] = Z["42"]*Z["24"] + (sp.Symbol("c^{42}")/g**2)*sp.eye(N) +  phi["43"]*phi["41"] - phi["41"]*phi["43"]
F["43"] = Z["43"]*Z["34"] + (sp.Symbol("c^{43}")/g**2)*sp.eye(N) -  phi["42"]*phi["41"] + phi["41"]*phi["42"]

## Writing in the G's

In [295]:
G = {}

G["21"] = phi_["12"]*Z["21"] + Z["23"]*Z["31"] + (Z["24"]*Z["41"])[0]
G["12"] = phi_["12"]*Z["12"] + Z["13"]*Z["32"] + (Z["14"]*Z["42"])[0]

G["31"] = phi_["31"]*Z["31"] + Z["32"]*Z["21"] - (Z["34"]*Z["41"])[0]
G["13"] = phi_["31"]*Z["13"] + Z["12"]*Z["23"] + (Z["14"]*Z["43"])[0]

G["32"] = phi_["23"]*Z["32"] + Z["31"]*Z["12"] + (Z["34"]*Z["42"])[0]
G["23"] = phi_["23"]*Z["23"] + Z["21"]*Z["13"] + (Z["24"]*Z["43"])[0]

G["41"] = -phi["41"]*Z["41"] + Z["42"]*Z["21"] + Z["43"]*Z["31"]
G["14"] = -Z["14"]*phi["41"] + Z["12"]*Z["24"] - Z["13"]*Z["34"]

G["42"] = -phi["42"]*Z["42"] + Z["43"]*Z["32"] + Z["41"]*Z["12"]
G["24"] = -Z["24"]*phi["42"] + Z["21"]*Z["14"] + Z["23"]*Z["34"]

G["43"] = -phi["43"]*Z["43"] - Z["41"]*Z["13"] + Z["42"]*Z["23"]
G["34"] = -Z["34"]*phi["43"] + Z["31"]*Z["14"] + Z["32"]*Z["24"]

## Creating $V_F$

In [303]:
start = time.time()
Fsum1 = sp.Symbol("",real = True)
Fsum2 = sp.Symbol("",real = True)
Fsum3 = sp.Symbol("",real = True)
#Term 1
for i in np.arange(1,4):
    for j in np.arange(1,4):
        if i == j:
            continue
        else:
            Fsum1+= sp.re(F[str(i)+str(j)])**2 + sp.im(F[str(i)+str(j)])**2
#Term 2
for i in np.arange(1,4):
    Fsum2 += sp.re(F[str(i)+"4"])**2 + sp.im(F[str(i)+"4"])**2
    
#Term 3
for i in np.arange(1,4):
    for j in np.arange(N):
        for k in np.arange(N):
            Fsum3 += sp.re(F["4"+str(i)][j,k])**2 + sp.im(F["4"+str(i)][j,k])**2
            
print(time.time() - start)

9.367836475372314


In [306]:
Fsum3 = sp.Symbol("")
for i in np.arange(1,4):
    Fsum3 += sp.trace(F["4"+str(i)].adjoint()*F["4"+str(i)])

In [313]:
(sp.trace(X["41"]*X["41"])).diff(X["41"])

2*X^{4}_1.T

In [321]:
sp.re(Z["43"][0])

P^{43}_1

In [316]:
Fsum3

 + Trace((-(1/2)*(Adjoint(X^{4}_4) - I*Adjoint(X^{4}_5))*(Adjoint(X^{4}_6) - I*Adjoint(X^{4}_7)) + (1/2)*(Adjoint(X^{4}_6) - I*Adjoint(X^{4}_7))*(Adjoint(X^{4}_4) - I*Adjoint(X^{4}_5)) + Matrix([
[(P^{34}_1 - I*Q^{34}_1)*(P^{43}_1 - I*Q^{43}_1) + conjugate(c^{43})/g**2,                          (P^{34}_1 - I*Q^{34}_1)*(P^{43}_2 - I*Q^{43}_2),                          (P^{34}_1 - I*Q^{34}_1)*(P^{43}_3 - I*Q^{43}_3)],
[                         (P^{34}_2 - I*Q^{34}_2)*(P^{43}_1 - I*Q^{43}_1), (P^{34}_2 - I*Q^{34}_2)*(P^{43}_2 - I*Q^{43}_2) + conjugate(c^{43})/g**2,                          (P^{34}_2 - I*Q^{34}_2)*(P^{43}_3 - I*Q^{43}_3)],
[                         (P^{34}_3 - I*Q^{34}_3)*(P^{43}_1 - I*Q^{43}_1),                          (P^{34}_3 - I*Q^{34}_3)*(P^{43}_2 - I*Q^{43}_2), (P^{34}_3 - I*Q^{34}_3)*(P^{43}_3 - I*Q^{43}_3) + conjugate(c^{43})/g**2]]))*((1/2)*(X^{4}_4 + I*X^{4}_5)*(X^{4}_6 + I*X^{4}_7) - (1/2)*(X^{4}_6 + I*X^{4}_7)*(X^{4}_4 + I*X^{4}_5) + Matrix([
[c^{43}/g**2 + (

In [319]:
Fsum3.diff(sp.re(Z["43"][0]))

NotImplementedError: 

In [310]:
Fsum3.diff(X["43"])

0

In [300]:
Fsum1

 + (P^{12}*P^{21} - Q^{12}*Q^{21} + re(c^{12})/g**2)**2 + (P^{12}*P^{21} - Q^{12}*Q^{21} + re(c^{21})/g**2)**2 + (P^{12}*Q^{21} + P^{21}*Q^{12} + im(c^{12})/g**2)**2 + (P^{12}*Q^{21} + P^{21}*Q^{12} + im(c^{21})/g**2)**2 + (P^{13}*P^{31} - Q^{13}*Q^{31} + re(c^{13})/g**2)**2 + (P^{13}*P^{31} - Q^{13}*Q^{31} + re(c^{31})/g**2)**2 + (P^{13}*Q^{31} + P^{31}*Q^{13} + im(c^{13})/g**2)**2 + (P^{13}*Q^{31} + P^{31}*Q^{13} + im(c^{31})/g**2)**2 + (P^{23}*P^{32} - Q^{23}*Q^{32} + re(c^{23})/g**2)**2 + (P^{23}*P^{32} - Q^{23}*Q^{32} + re(c^{32})/g**2)**2 + (P^{23}*Q^{32} + P^{32}*Q^{23} + im(c^{23})/g**2)**2 + (P^{23}*Q^{32} + P^{32}*Q^{23} + im(c^{32})/g**2)**2

In [297]:
V_F = 2*(Fsum1 + Fsum2 + Fsum3)
V_F.diff(sp.re(Z["41"]))

Matrix([
[4*P^{14}_1*(P^{14}_1*P^{41}_1 + P^{14}_2*P^{41}_2 + P^{14}_3*P^{41}_3 - Q^{14}_1*Q^{41}_1 - Q^{14}_2*Q^{41}_2 - Q^{14}_3*Q^{41}_3 + 3*re(c^{14})/g**2) + 4*P^{14}_1*(P^{14}_1*P^{41}_1 - Q^{14}_1*Q^{41}_1 + re(X^{4}_6[0, 1]*X^{4}_8[1, 0])/2 + re(X^{4}_6[0, 2]*X^{4}_8[2, 0])/2 - re(X^{4}_6[1, 0]*X^{4}_8[0, 1])/2 - re(X^{4}_6[2, 0]*X^{4}_8[0, 2])/2 - re(X^{4}_7[0, 1]*X^{4}_9[1, 0])/2 - re(X^{4}_7[0, 2]*X^{4}_9[2, 0])/2 + re(X^{4}_7[1, 0]*X^{4}_9[0, 1])/2 + re(X^{4}_7[2, 0]*X^{4}_9[0, 2])/2 - im(X^{4}_6[0, 1]*X^{4}_9[1, 0])/2 - im(X^{4}_6[0, 2]*X^{4}_9[2, 0])/2 + im(X^{4}_6[1, 0]*X^{4}_9[0, 1])/2 + im(X^{4}_6[2, 0]*X^{4}_9[0, 2])/2 - im(X^{4}_7[0, 1]*X^{4}_8[1, 0])/2 - im(X^{4}_7[0, 2]*X^{4}_8[2, 0])/2 + im(X^{4}_7[1, 0]*X^{4}_8[0, 1])/2 + im(X^{4}_7[2, 0]*X^{4}_8[0, 2])/2 + re(c^{41})/g**2) + 4*P^{14}_2*(P^{14}_2*P^{41}_1 - Q^{14}_2*Q^{41}_1 + re(X^{4}_6[0, 0]*X^{4}_8[0, 1])/2 - re(X^{4}_6[0, 1]*X^{4}_8[0, 0])/2 + re(X^{4}_6[0, 1]*X^{4}_8[1, 1])/2 + re(X^{4}_6[0, 2]*X^{4}_8[2, 1]

In [298]:
V_F.diff(sp.re(Z["41"][0]))

4*P^{14}_1*(P^{14}_1*P^{41}_1 + P^{14}_2*P^{41}_2 + P^{14}_3*P^{41}_3 - Q^{14}_1*Q^{41}_1 - Q^{14}_2*Q^{41}_2 - Q^{14}_3*Q^{41}_3 + 3*re(c^{14})/g**2) + 4*P^{14}_1*(P^{14}_1*P^{41}_1 - Q^{14}_1*Q^{41}_1 + re(X^{4}_6[0, 1]*X^{4}_8[1, 0])/2 + re(X^{4}_6[0, 2]*X^{4}_8[2, 0])/2 - re(X^{4}_6[1, 0]*X^{4}_8[0, 1])/2 - re(X^{4}_6[2, 0]*X^{4}_8[0, 2])/2 - re(X^{4}_7[0, 1]*X^{4}_9[1, 0])/2 - re(X^{4}_7[0, 2]*X^{4}_9[2, 0])/2 + re(X^{4}_7[1, 0]*X^{4}_9[0, 1])/2 + re(X^{4}_7[2, 0]*X^{4}_9[0, 2])/2 - im(X^{4}_6[0, 1]*X^{4}_9[1, 0])/2 - im(X^{4}_6[0, 2]*X^{4}_9[2, 0])/2 + im(X^{4}_6[1, 0]*X^{4}_9[0, 1])/2 + im(X^{4}_6[2, 0]*X^{4}_9[0, 2])/2 - im(X^{4}_7[0, 1]*X^{4}_8[1, 0])/2 - im(X^{4}_7[0, 2]*X^{4}_8[2, 0])/2 + im(X^{4}_7[1, 0]*X^{4}_8[0, 1])/2 + im(X^{4}_7[2, 0]*X^{4}_8[0, 2])/2 + re(c^{41})/g**2) + 4*P^{14}_2*(P^{14}_2*P^{41}_1 - Q^{14}_2*Q^{41}_1 + re(X^{4}_6[0, 0]*X^{4}_8[0, 1])/2 - re(X^{4}_6[0, 1]*X^{4}_8[0, 0])/2 + re(X^{4}_6[0, 1]*X^{4}_8[1, 1])/2 + re(X^{4}_6[0, 2]*X^{4}_8[2, 1])/2 - re(X

In [156]:
(sp.trace(X["41"].adjoint()*X["41"])).diff(X["41"][0,0])

2*x^{41}_{11}

In [257]:
col = []
for i in np.arange(N):
    row = []
    for j in np.arange(N):
        row.append(sp.trace(X["41"].adjoint()*X["41"]).diff(sp.re(X["41"][i,j])))
    col.append(row)
ba = sp.Matrix(col)

In [261]:
sp.trace(X["41"].adjoint()*X["41"]).expand()

x^{41}_{11}**2 + 2*x^{41}_{12}**2 + 2*x^{41}_{13}**2 + x^{41}_{22}**2 + 2*x^{41}_{23}**2 + x^{41}_{33}**2 + 2*y^{41}_{12}**2 + 2*y^{41}_{13}**2 + 2*y^{41}_{23}**2

In [269]:
sp.trace(Z["14"].adjoint()*Z["14"]).diff(sp.re(Z["14"]))

Matrix([[2*P^{14}_1, 2*P^{14}_2, 2*P^{14}_3]])

In [243]:
col = []
for i in np.arange(N):
    row = []
    for j in np.arange(N):
        if i < j:
            row.append(sp.trace(X["41"].adjoint()*X["41"]).diff(sp.im(X["41"][i,j])))
        elif i > j:
            row.append(sp.trace(X["41"].adjoint()*X["41"]).diff(-sp.im(X["41"][i,j])))
        else:
            row.append(0)
    col.append(row)
jah = sp.Matrix(col)

In [281]:
Z14 = sp.MatrixSymbol("Z^{14}",1,N)